In [49]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [50]:
def from_file(filename):
    cap = cv2.VideoCapture(filename)
    ret, frame = cap.read()
    if not ret:
        raise ValueError("Video stream not opened")
    prev = frame.copy()
    while ret:
        key = cv2.waitKey(20) & 0xff
        if key == 27 or not ret:
            break
        cv2.imshow('ORIGINAL', frame)
        cv2.imshow('MOVEMENT', movement_tracker(frame, prev))
        prev = frame.copy()
        ret, frame = cap.read()
    
    cv2.destroyAllWindows()
    cap.release()

In [51]:
def movement_tracker(fst, snd):
    frame_diff = cv2.absdiff(fst, snd)
    frame_diff = cv2.GaussianBlur(cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY), (5,5), 0)
    frame_diff  = cv2.dilate(frame_diff, None, iterations=3)
    frame_diff = cv2.threshold(frame_diff, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(frame_diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    out_frame   = snd.copy()
    movement_marks = np.empty_like(out_frame, np.uint8)
    for con in contours:
        (x,y,w,h) = cv2.boundingRect(con)
        if cv2.contourArea(con) > 120:
            cv2.rectangle(movement_marks, (x,y), (x+w, y+h), (0,0,255), cv2.FILLED)
    mask = movement_marks.astype(bool)
    out_frame[mask] = cv2.addWeighted(out_frame, 0.5, movement_marks, 0.5, 0)[mask]
    return out_frame

In [52]:
def movement_tracker_TWO(fst, snd):
    #Work with image to find movements
    frame_diff = cv2.absdiff(fst, snd)
    frame_diff = cv2.medianBlur(cv2.cvtColor(frame_diff, cv2.COLOR_BGR2GRAY), 5) #Блюр чтобы сгладить шум
    frame_diff = cv2.threshold(frame_diff, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1] #Если значение пиксели выше thresh то он белый, остальные в черный
    frame_diff = cv2.dilate(frame_diff, np.ones((3,3)), iterations=3) #Расширить белые пиксели

    frame_diff_color = cv2.cvtColor(frame_diff, cv2.COLOR_GRAY2BGR)
    frame_diff_color[np.where((frame_diff_color==[255, 255, 255]).all(axis=2))] = [0, 0, 255] #Color white to red
    frame_diff_color[np.where((frame_diff_color==[0, 0, 0]).all(axis=2))] = [0, 255, 0] #Color black to GREEN
    alpha = 0.7
    out_frame = cv2.addWeighted(fst, alpha, frame_diff_color, 1-alpha, 0)
    return out_frame

In [53]:
from vidgear.gears import CamGear
def from_site(url):
    options = {"STREAM_RESOLUTION": "480p", }
    stream = CamGear(source=url, stream_mode=True,logging=False, **options).start()
    prev = stream.read()
    while True:
        key = cv2.waitKey(1) & 0xFF
        frame = stream.read()
        if frame is None or key == 27: break #ESC
        #cv2.imshow('ORIGINAL', frame)
        cv2.imshow('MOVEMENTTWO', movement_tracker_TWO(frame, prev))
        #cv2.imshow('MOVEMENT', movement_tracker(frame, prev))
        prev = frame.copy()
        
    cv2.destroyAllWindows()
    stream.stop()


In [54]:
if __name__ == '__main__':
    #from_file("Coffin Dance (Official Music Video HD).mp4")
    from_site("https://www.youtube.com/watch?v=akjT10sjPTc")
    #https://www.youtube.com/watch?v=cIvkWkQSpq4
    #https://www.youtube.com/watch?v=rG13FY2ytno
    #"https://www.youtube.com/watch?v=akjT10sjPTc"

15:52:37 ::    Helper     :: WARNING  :: GStreamer not found!
15:52:37 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
15:52:38 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, 1080p, best, worst]
15:52:38 ::    CamGear    :: WARNING  :: Livestream URL detected. It is advised to use GStreamer backend(`cv2.CAP_GSTREAMER`) with it.
